<a href="https://colab.research.google.com/github/mshitie2/main/blob/main/L_CBAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Google Drive Permissions
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rarfile

from google.colab import drive
import rarfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to the .rar file in Google Drive
rar_path = '/content/drive/MyDrive/Computer Vision/main_data_croped.rar'

# Destination folder to extract the contents
destination_folder = '/content'

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Extract the .rar file
with rarfile.RarFile(rar_path, 'r') as rar_ref:
    rar_ref.extractall(destination_folder)

print("Extraction completed.")

In [ ]:
#convolutional block attention module (CBAM) with  Apply Lambda layer for rescaling the attention results, VGG16
!pip install tensorflow-addons
import os
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Conv2D, Lambda, Reshape, GlobalMaxPooling2D, Average, Multiply, Concatenate, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix

# Set the path to the directory containing the dataset
dataset_dir = '/content/main_data_croped'

# Read the CSV file
data = pd.read_csv(os.path.join(dataset_dir, '/content/drive/MyDrive/Computer Vision/tfti2.csv'), usecols=["key", "class"])

# Convert the 'class' column to string
data['class'] = data['class'].astype(str)

# Filter the data to include only classes 1 and 2
data = data[data['class'].isin(['1', '2', '3'])]

# Split the data into training, validation, and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=0)

# Print the number of samples in each set
print('Number of train samples:', train_data.shape[0])
print('Number of valid samples:', valid_data.shape[0])
print('Number of test samples:', test_data.shape[0])

# Preprocess data
train_data["key"] = train_data["key"].apply(lambda x: x + ".jpg")
valid_data["key"] = valid_data["key"].apply(lambda x: x + ".jpg")
test_data["key"] = test_data["key"].apply(lambda x: x + ".jpg")

BATCH_SIZE = 16
HEIGHT = 224
WIDTH = 224
N_CLASSES = 3

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=dataset_dir,
    x_col="key",
    y_col="class",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    target_size=(HEIGHT, WIDTH),
    seed=0
)

valid_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_data,
    directory=dataset_dir,
    x_col="key",
    y_col="class",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    target_size=(HEIGHT, WIDTH),
    seed=0
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=dataset_dir,
    x_col="key",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    target_size=(HEIGHT, WIDTH),
    seed=0
)

# Load the VGG16 model
vgg_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(HEIGHT, WIDTH, 3)
)

# Channel attention module
x = vgg_model.output
avg_pool = GlobalAveragePooling2D()(x)
max_pool = GlobalMaxPooling2D()(x)
avg_pool = Dense(512)(avg_pool)
max_pool = Dense(512)(max_pool)
avg_pool = Activation('relu')(avg_pool)
max_pool = Activation('relu')(max_pool)
channel_attention = Average()([avg_pool, max_pool])
channel_attention = Dense(512)(channel_attention)
channel_attention = Activation('sigmoid')(channel_attention)
channel_attention = Reshape((1, 1, 512))(channel_attention)
channel_attention = Lambda(lambda x: x / tf.reduce_max(x))(channel_attention)
x = Multiply()([x, channel_attention])

# Spatial attention module
avg_pool = Lambda(lambda x: tf.keras.backend.mean(x, axis=3, keepdims=True))(x)
max_pool = Lambda(lambda x: tf.keras.backend.max(x, axis=3, keepdims=True))(x)
concat = Concatenate(axis=3)([avg_pool, max_pool])
spatial_attention = Conv2D(filters=1, kernel_size=7, padding='same', activation='sigmoid')(concat)
#spatial_attention = Lambda(lambda x: x / tf.reduce_max(x))(spatial_attention)
x = Multiply()([x, spatial_attention])

# Build the model architecture
input_tensor = Input(shape=(HEIGHT, WIDTH, 3))
x = vgg_model(input_tensor)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout regularization
x = Dense(N_CLASSES, activation='softmax', kernel_regularizer=l2(0.01), name='output')(x)
model_vgg3 = Model(inputs=input_tensor, outputs=x)


# Compile the model
model_vgg3.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy', # binary_crossentropy for two class
    metrics=['accuracy']
)

# Learning Rate Scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=1e-7)

# Fine-tune the model
EPOCHS = 50
history_model3 = model_vgg3.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    callbacks=[reduce_lr],
    verbose=2
)

# Evaluate the model on the test set
test_loss, test_acc = model_vgg3.evaluate(
    test_generator,
    verbose=0
)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import pandas as pd

# Extract features using pre-trained model
predictions = model_vgg3.layers[-2].output
model_feat = keras.Model(inputs=model_vgg3.inputs, outputs=predictions)
Extracted_features = model_feat.predict(test_generator)

# Split data into training and testing sets
train_image, valid_image, train_label, valid_label = train_test_split(Extracted_features, test_generator.classes,test_size=0.20, stratify=test_generator.classes, random_state=10)
val_img, test_img, val_lab, test_lab = train_test_split(valid_image, valid_label, test_size=0.5, stratify=valid_label, random_state=50)

# Train MLP classifier
_classifier = MLPClassifier(hidden_layer_sizes=(64,), activation='relu', solver='adam', max_iter=1000)
_classifier.fit(train_image, train_label)

# Evaluate performance on test data
pred = _classifier.predict(test_img)
acc = accuracy_score(y_true=test_lab, y_pred=pred)
print('Test accuracy:', acc)

# Save extracted features and labels as CSV files
final_features = Extracted_features
lab = np.array(test_generator.classes)
lab = lab.reshape(lab.shape[0], 1)
np.savetxt("ext_features.csv", final_features, delimiter=",")
np.savetxt("labels.csv", lab, delimiter=",")

# Load and prepare data for training a new model
df = pd.read_csv('/content/ext_features.csv', header=None)
df2 = pd.read_csv('/content/labels.csv', header=None)

total_features = df.shape[1]
x = df[df.columns[:total_features]]
y = df2[df2.columns[-1]].astype(int)

# Split data into training and testing sets
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, stratify=y, random_state=10)

print('Training data shape:',train_x.shape)
print('Training labels shape:', train_y.shape)
print('Testing data shape:', test_x.shape)
print('Testing labels shape:', test_y.shape)

In [ ]:
#SSD without ABHC
import numpy as np
import pandas as pd
import math
import random
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

swarm_size = 20   #population size
max_iterations = 100
omega = 0.2  #used in the fitness function
delta=0.2   #to set an upper limit for including a slightly worse particle in LAHC

def mutate(agent):
    percent=0.2
    numChange=int(total_features*percent)
    pos=np.random.randint(0,total_features-1,numChange) #choose random positions to be mutated
    agent[pos]=1-agent[pos] #mutation
    return agent

def find_fitness(particle):
    features = []
    for x in range(len(particle)):
        if particle[x]>=0.5: #convert it to zeros and ones
            features.append(df.columns[x])
    if(len(features)==0):
        return 10000
    new_x_train = train_x[features].copy()
    new_x_test = test_x[features].copy()

    _classifier = KNeighborsClassifier(n_neighbors=5)
    _classifier.fit(new_x_train, train_y)
    predictions = _classifier.predict(new_x_test)
    acc = accuracy_score(y_true = test_y, y_pred = predictions)
    fitness = acc
    err=1-acc
    num_features = len(features)
    fitness =  alpha*err + (1-alpha)*(num_features/total_features)

    return fitness

def transfer_func(velocity): #to convert into an array of zeros and ones
    t=[]
    for i in range(len(velocity)):
        t.append(abs(velocity[i]/(math.sqrt(1+velocity[i]*velocity[i])))) #transfer function inside paranthesis
    return t

#initialize swarm position and swarm velocity of SSD
swarm_vel = np.random.uniform(low=0, high=1, size=(swarm_size,total_features))

swarm_pos = np.random.uniform(size=(swarm_size,total_features))
swarm_pos = np.where(swarm_pos>=0.5,1,0)

c = 100
alpha= 0.9

gbest_fitness=100000
pbest_fitness = np.zeros(swarm_size)
pbest_fitness.fill(np.inf)  #initialize with the worse possible values
pbest = np.empty((swarm_size,total_features))
gbest = np.empty(total_features)
pbest.fill(np.inf)
gbest.fill(np.inf)

for itr in range(max_iterations):

    for i in range(swarm_size):

        fitness = find_fitness(swarm_pos[i])

        if fitness < gbest_fitness:

            gbest=swarm_pos[i].copy() #updating global best
            gbest_fitness=fitness

        if fitness < pbest_fitness[i]:
            pbest[i] = swarm_pos[i].copy() #updating personal best
            pbest_fitness[i]=fitness

        r1 = random.random()
        r2 = random.random()

        #updating the swarm velocity
        if r1 < 0.5:
            swarm_vel[i] = c*math.sin(r2)*(pbest[i]-swarm_pos[i]) +math.sin(r2)* (gbest-swarm_pos[i])
        else:
            swarm_vel[i] = c*math.cos(r2)*(pbest[i]-swarm_pos[i]) + math.cos(r2)*(gbest-swarm_pos[i])

        #decaying value of c
        alpha= 0.9
        c=alpha*c;

        #applying transfer function and then updating the swarm position
        t = transfer_func(swarm_vel[i])
        for j in range(len(swarm_pos[i])):
            if(t[j] < 0.5):
                swarm_pos[i][j] = swarm_pos[i][j]
            else:
                swarm_pos[i][j] = 1 - swarm_pos[i][j]

selected_features = gbest
print(gbest_fitness)

number_of_selected_features = np.sum(selected_features)
print("#",number_of_selected_features)

features=[]
for j in range(len(selected_features)):
    if selected_features[j]==1:
        features.append(df.columns[j])
new_x_train = train_x[features]
new_x_test = test_x[features]

_classifier = KNeighborsClassifier(n_neighbors=5)
_classifier.fit(new_x_train, train_y)
predictions = _classifier.predict(new_x_test)
acc = accuracy_score(y_true = test_y, y_pred = predictions)
pre = precision_score(y_true = test_y, y_pred = predictions,average=None)
rec = recall_score(y_true = test_y, y_pred = predictions, average=None)
result = classification_report(y_true=test_y, y_pred=predictions)

print("Accuracy: ", acc)
print("Precision: ", pre)
print("Recall: ", rec)
print("Classification Report: \n", result)

In [ ]:
# using ABHC
import numpy as np
import pandas as pd
import math
import random
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

swarm_size = 20   #population size
max_iterations = 100
omega = 0.2  #used in the fitness function
delta=0.2   #to set an upper limit for including a slightly worse particle in LAHC


def mutate(agent):
    percent=0.2
    numChange=int(total_features*percent)
    pos=np.random.randint(0,total_features-1,numChange) #choose random positions to be mutated
    agent[pos]=1-agent[pos] #mutation
    return agent

def ABHC(particle):
    _lambda = 15 #upper limit on number of iterations in LAHC
    target_fitness = find_fitness(particle) #original fitness
    for i in range(_lambda):
        new_particle = mutate(particle) #first mutation
        temp = find_fitness(new_particle)
        if temp < target_fitness:
            particle = new_particle.copy() #updation
            target_fitness = temp
        elif (temp<=(1+delta)*target_fitness):
            temp_particle = new_particle.copy()
            for j in range(_lambda):
                temp_particle1 = mutate(temp_particle) #second mutation
                temp_fitness = find_fitness(temp_particle1)
                if temp_fitness < target_fitness:
                    target_fitness=temp_fitness
                    particle=temp_particle1.copy() #updation
                break
    return particle

def randomwalk(agent):
    percent = 30
    percent /= 100
    neighbor = agent.copy()
    size = np.shape(agent)[0]
    upper = int(percent*size)
    if upper <= 1:
        upper = size
    x = random.randint(1,upper)
    pos = random.sample(range(0,size - 1),x)
    for i in pos:
        neighbor[i] = 1 - neighbor[i]
    return neighbor

def adaptiveBeta(agent):
    bmin = 0.1 #parameter: (can be made 0.01)
    bmax = 1
    maxIter = 10 # parameter: (can be increased )

    agentFit = find_fitness(agent)
    for curr in range(maxIter):
        neighbor = agent.copy()
        size = np.shape(neighbor)[0]
        neighbor = randomwalk(neighbor)

        beta = bmin + (curr / maxIter)*(bmax - bmin)
        for i in range(size):
            random.seed( time.time() + i )
            if random.random() <= beta:
                neighbor[i] = agent[i].copy()
        neighFit = find_fitness(neighbor)
        if neighFit <= agentFit:
            agent = neighbor.copy()

    return agent

def find_fitness(particle):
    features = []
    for x in range(len(particle)):
        if particle[x]>=0.5: #convert it to zeros and ones
            features.append(df.columns[x])
    if(len(features)==0):
        return 10000
    new_x_train = train_x[features].copy()
    new_x_test = test_x[features].copy()

    _classifier = KNeighborsClassifier(n_neighbors=5)
    _classifier.fit(new_x_train, train_y)
    predictions = _classifier.predict(new_x_test)
    acc = accuracy_score(y_true = test_y, y_pred = predictions)
    fitness = acc
    err=1-acc
    num_features = len(features)
    fitness =  alpha*err + (1-alpha)*(num_features/total_features)

    return fitness

def transfer_func(velocity): #to convert into an array of zeros and ones
    t=[]
    for i in range(len(velocity)):
        t.append(abs(velocity[i]/(math.sqrt(1+velocity[i]*velocity[i])))) #transfer function inside paranthesis
    return t

#initialize swarm position and swarm velocity of SSD
swarm_vel = np.random.uniform(low=0, high=1, size=(swarm_size,total_features))

swarm_pos = np.random.uniform(size=(swarm_size,total_features))
swarm_pos = np.where(swarm_pos>=0.5,1,0)

c = 100
alpha= 0.9

gbest_fitness=100000
pbest_fitness = np.zeros(swarm_size)
pbest_fitness.fill(np.inf)  #initialize with the worse possible values
pbest = np.empty((swarm_size,total_features))
gbest = np.empty(total_features)
pbest.fill(np.inf)
gbest.fill(np.inf)

for itr in range(max_iterations):

    for i in range(swarm_size):

        swarm_pos[i] = adaptiveBeta(swarm_pos[i]) #for ABHC local search
        #swarm_pos[i] = LAHC(swarm_pos[i]) #for LAHC local search
        fitness = find_fitness(swarm_pos[i])

        if fitness < gbest_fitness:

            gbest=swarm_pos[i].copy() #updating global best
            gbest_fitness=fitness

        if fitness < pbest_fitness[i]:
            pbest[i] = swarm_pos[i].copy() #updating personal best
            pbest_fitness[i]=fitness

        r1 = random.random()
        r2 = random.random()

        #updating the swarm velocity
        if r1 < 0.5:
            swarm_vel[i] = c*math.sin(r2)*(pbest[i]-swarm_pos[i]) +math.sin(r2)* (gbest-swarm_pos[i])
        else:
            swarm_vel[i] = c*math.cos(r2)*(pbest[i]-swarm_pos[i]) + math.cos(r2)*(gbest-swarm_pos[i])

        #decaying value of c
        alpha= 0.9
        c=alpha*c;

        #applying transfer function and then updating the swarm position
        t = transfer_func(swarm_vel[i])
        for j in range(len(swarm_pos[i])):
            if(t[j] < 0.5):
                swarm_pos[i][j] = swarm_pos[i][j]
            else:
                swarm_pos[i][j] = 1 - swarm_pos[i][j]

selected_features = gbest
print(gbest_fitness)

number_of_selected_features = np.sum(selected_features)
print("#",number_of_selected_features)

features=[]
for j in range(len(selected_features)):
    if selected_features[j]==1:
        features.append(df.columns[j])
new_x_train = train_x[features]
new_x_test = test_x[features]

_classifier = KNeighborsClassifier(n_neighbors=5)
_classifier.fit(new_x_train, train_y)
predictions = _classifier.predict(new_x_test)
acc = accuracy_score(y_true = test_y, y_pred = predictions)
pre = precision_score(y_true = test_y, y_pred = predictions,average=None)
rec = recall_score(y_true = test_y, y_pred = predictions, average=None)
result = classification_report(y_true = test_y, y_pred = predictions, digits=5)
fitness = acc
print("Acc:",fitness)
print("Precision:", pre)
print("Recall:",rec)
print(result)
print("\n\n")

In [ ]:
# using LAHC
import numpy as np
import pandas as pd
import math
import random
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

swarm_size = 20   #population size
max_iterations = 100
omega = 0.2  #used in the fitness function
delta=0.2   #to set an upper limit for including a slightly worse particle in LAHC


def mutate(agent):
    percent=0.2
    numChange=int(total_features*percent)
    pos=np.random.randint(0,total_features-1,numChange) #choose random positions to be mutated
    agent[pos]=1-agent[pos] #mutation
    return agent

def LAHC(particle):
    _lambda = 15 #upper limit on number of iterations in LAHC
    target_fitness = find_fitness(particle) #original fitness
    for i in range(_lambda):
        new_particle = mutate(particle) #first mutation
        temp = find_fitness(new_particle)
        if temp < target_fitness:
            particle = new_particle.copy() #updation
            target_fitness = temp
        elif (temp<=(1+delta)*target_fitness):
            temp_particle = new_particle.copy()
            for j in range(_lambda):
                temp_particle1 = mutate(temp_particle) #second mutation
                temp_fitness = find_fitness(temp_particle1)
                if temp_fitness < target_fitness:
                    target_fitness=temp_fitness
                    particle=temp_particle1.copy() #updation
                break
    return particle

def randomwalk(agent):
    percent = 30
    percent /= 100
    neighbor = agent.copy()
    size = np.shape(agent)[0]
    upper = int(percent*size)
    if upper <= 1:
        upper = size
    x = random.randint(1,upper)
    pos = random.sample(range(0,size - 1),x)
    for i in pos:
        neighbor[i] = 1 - neighbor[i]
    return neighbor

def adaptiveBeta(agent):
    bmin = 0.1 #parameter: (can be made 0.01)
    bmax = 1
    maxIter = 10 # parameter: (can be increased )

    agentFit = find_fitness(agent)
    for curr in range(maxIter):
        neighbor = agent.copy()
        size = np.shape(neighbor)[0]
        neighbor = randomwalk(neighbor)

        beta = bmin + (curr / maxIter)*(bmax - bmin)
        for i in range(size):
            random.seed( time.time() + i )
            if random.random() <= beta:
                neighbor[i] = agent[i].copy()
        neighFit = find_fitness(neighbor)
        if neighFit <= agentFit:
            agent = neighbor.copy()

    return agent

def find_fitness(particle):
    features = []
    for x in range(len(particle)):
        if particle[x]>=0.5: #convert it to zeros and ones
            features.append(df.columns[x])
    if(len(features)==0):
        return 10000
    new_x_train = train_x[features].copy()
    new_x_test = test_x[features].copy()

    _classifier = KNeighborsClassifier(n_neighbors=5)
    _classifier.fit(new_x_train, train_y)
    predictions = _classifier.predict(new_x_test)
    acc = accuracy_score(y_true = test_y, y_pred = predictions)
    fitness = acc
    err=1-acc
    num_features = len(features)
    fitness =  alpha*err + (1-alpha)*(num_features/total_features)

    return fitness

def transfer_func(velocity): #to convert into an array of zeros and ones
    t=[]
    for i in range(len(velocity)):
        t.append(abs(velocity[i]/(math.sqrt(1+velocity[i]*velocity[i])))) #transfer function inside paranthesis
    return t

#initialize swarm position and swarm velocity of SSD
swarm_vel = np.random.uniform(low=0, high=1, size=(swarm_size,total_features))

swarm_pos = np.random.uniform(size=(swarm_size,total_features))
swarm_pos = np.where(swarm_pos>=0.5,1,0)

c = 100
alpha= 0.9

gbest_fitness=100000
pbest_fitness = np.zeros(swarm_size)
pbest_fitness.fill(np.inf)  #initialize with the worse possible values
pbest = np.empty((swarm_size,total_features))
gbest = np.empty(total_features)
pbest.fill(np.inf)
gbest.fill(np.inf)

for itr in range(max_iterations):

    for i in range(swarm_size):

        #swarm_pos[i] = adaptiveBeta(swarm_pos[i]) #for ABHC local search
        swarm_pos[i] = LAHC(swarm_pos[i]) #for LAHC local search
        fitness = find_fitness(swarm_pos[i])

        if fitness < gbest_fitness:

            gbest=swarm_pos[i].copy() #updating global best
            gbest_fitness=fitness

        if fitness < pbest_fitness[i]:
            pbest[i] = swarm_pos[i].copy() #updating personal best
            pbest_fitness[i]=fitness

        r1 = random.random()
        r2 = random.random()

        #updating the swarm velocity
        if r1 < 0.5:
            swarm_vel[i] = c*math.sin(r2)*(pbest[i]-swarm_pos[i]) +math.sin(r2)* (gbest-swarm_pos[i])
        else:
            swarm_vel[i] = c*math.cos(r2)*(pbest[i]-swarm_pos[i]) + math.cos(r2)*(gbest-swarm_pos[i])

        #decaying value of c
        alpha= 0.9
        c=alpha*c;

        #applying transfer function and then updating the swarm position
        t = transfer_func(swarm_vel[i])
        for j in range(len(swarm_pos[i])):
            if(t[j] < 0.5):
                swarm_pos[i][j] = swarm_pos[i][j]
            else:
                swarm_pos[i][j] = 1 - swarm_pos[i][j]

selected_features = gbest
print(gbest_fitness)

number_of_selected_features = np.sum(selected_features)
print("#",number_of_selected_features)

features=[]
for j in range(len(selected_features)):
    if selected_features[j]==1:
        features.append(df.columns[j])
new_x_train = train_x[features]
new_x_test = test_x[features]

_classifier = KNeighborsClassifier(n_neighbors=5)
_classifier.fit(new_x_train, train_y)
predictions = _classifier.predict(new_x_test)
acc = accuracy_score(y_true = test_y, y_pred = predictions)
pre = precision_score(y_true = test_y, y_pred = predictions,average=None)
rec = recall_score(y_true = test_y, y_pred = predictions, average=None)
result = classification_report(y_true = test_y, y_pred = predictions, digits=5)
fitness = acc
print("Acc:",fitness)
print("Precision:", pre)
print("Recall:",rec)
print(result)
print("\n\n")

In [ ]:
!pip install nbconvert

In [ ]:
!jupyter nbconvert --to=python Comparison_algorithms/GA.ipynb # first upload EO.ipynb in Colab environment in Comparison_algorithms folder
!jupyter nbconvert --to=python Comparison_algorithms/GSA.ipynb
!jupyter nbconvert --to=python Comparison_algorithms/GWO.ipynb
!jupyter nbconvert --to=python Comparison_algorithms/WOA.ipynb
!jupyter nbconvert --to=python Comparison_algorithms/PSO.ipynb
!jupyter nbconvert --to=python Comparison_algorithms/SCA.ipynb
!jupyter nbconvert --to=python Comparison_algorithms/EO.ipynb
!jupyter nbconvert --to=python Comparison_algorithms/HS.ipynb


In [ ]:
from Comparison_algorithms.GA import GA # for error copy below confusion matric code and pased by oping link and poast on utilities.py
from Comparison_algorithms.GSA import GSA
from Comparison_algorithms.GWO import GWO
from Comparison_algorithms.WOA import WOA
from Comparison_algorithms.PSO import PSO
from Comparison_algorithms.SCA import SCA
from Comparison_algorithms.EO import EO
from Comparison_algorithms.HS import HS

In [ ]:
#solution, result = GA(20, 100, x, y, save_conv_graph=True)   # To run Local search based GA algorithm
#solution, result = GSA(20, 100, x, y, save_conv_graph=True)  # To run Local search based GSA algorithm
#solution, result = GWO(20, 100, x, y, save_conv_graph=True)  # To run Local search based GWO algorithm
#solution, result = WOA(20, 100, x, y, save_conv_graph=True)   # To run Local search based WOA algorithm
#solution, result = PSO(20, 100, x, y, save_conv_graph=True)  # To run Local search based PSO algorithm
#solution, result = SCA(20, 100, x, y, save_conv_graph=True)  # To run Local search based SCA algorithm
solution, result = EO(20, 100, x, y, save_conv_graph=True)   # To run Local search based EO algorithm
#1solution, result = HS(20, 100, x, y, save_conv_graph=True)   # To run Local search based HS algorithm
print(result.accuracy)
print(result.precision)
print(result.recall)